In [1]:
import numpy as np
import random
import sys
import io
import os
from mb_utils import *
%matplotlib inline
import pickle

import pandas as pd

c:\users\lukewolc\appdata\local\programs\python\python37\lib\site-packages\pydub\utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
Using TensorFlow backend.


In [2]:
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape, Flatten
from keras.optimizers import Adam
from keras.utils import np_utils
import keras.backend as K
from time import time

In [3]:
from importlib import reload
import mb_utils

In [4]:
reload(mb_utils)
from mb_utils import *

In [5]:
Tx = 2754 #5511 # The number of time steps input to the model from the spectrogram
n_freq = 101 # Number of frequencies input to the model at each time step of the spectrogram
Ty = 1375 # The number of time steps in the output of our model

# model training

In [6]:
# load training features and labels for 1000 

file = open('XY_train/X_1000_5sec_v0.pkl', 'rb')
X_training = pickle.load(file)
file.close()

file = open('XY_train/y_1000_5sec_v0.pkl', 'rb')
y_training = pickle.load(file)
file.close()

# load validation features and labels
file = open('XY_val/X_400_5sec_100_100_200.pkl', 'rb')
X_val = pickle.load(file)
file.close()

file = open('XY_val/y_400_5sec_100_100_200.pkl', 'rb')
y_val = pickle.load(file)
file.close()

# one-hot encode labels
y_training_oh = encode_and_oh_labels(y_training)
y_val_oh = encode_and_oh_labels(y_val)

print('Size of X_training: {}'.format(X_training.shape))
print('Size of X_val: {}'.format(X_val.shape))

Size of X_training: (1000, 2754, 101)
Size of X_val: (400, 2754, 101)


In [6]:
# load training features and labels for 4000 over 4 files

#v0
file = open('XY_train/X_1000_5sec_v0.pkl', 'rb')
X_training_v0 = pickle.load(file)
file.close()

file = open('XY_train/y_1000_5sec_v0.pkl', 'rb')
y_training_v0 = pickle.load(file)
file.close()

#v1
file = open('XY_train/X_1000_5sec_v1.pkl', 'rb')
X_training_v1 = pickle.load(file)
file.close()

file = open('XY_train/y_1000_5sec_v1.pkl', 'rb')
y_training_v1 = pickle.load(file)
file.close()

#v2
file = open('XY_train/X_1000_5sec_v2.pkl', 'rb')
X_training_v2 = pickle.load(file)
file.close()

file = open('XY_train/y_1000_5sec_v2.pkl', 'rb')
y_training_v2 = pickle.load(file)
file.close()

#v3
file = open('XY_train/X_1000_5sec_v3.pkl', 'rb')
X_training_v3 = pickle.load(file)
file.close()

file = open('XY_train/y_1000_5sec_v3.pkl', 'rb')
y_training_v3 = pickle.load(file)
file.close()

X_training = np.concatenate([X_training_v0, X_training_v1, X_training_v2, X_training_v3])
y_training = np.concatenate([y_training_v0, y_training_v1, y_training_v2, y_training_v3])

del(X_training_v0)
del(X_training_v1)
del(X_training_v2)
del(X_training_v3)

# load validation features and labels
file = open('XY_val/X_400_5sec_100_100_200.pkl', 'rb')
X_val = pickle.load(file)
file.close()

file = open('XY_val/y_400_5sec_100_100_200.pkl', 'rb')
y_val = pickle.load(file)
file.close()

# one-hot encode labels
y_training_oh = encode_and_oh_labels(y_training)
y_val_oh = encode_and_oh_labels(y_val)

print('Size of X_training: {}'.format(X_training.shape))
print('Size of X_val: {}'.format(X_val.shape))

Size of X_training: (4000, 2754, 101)
Size of X_val: (400, 2754, 101)


In [6]:
# load training features and labels for 8000 over 8 files

#v0
file = open('XY_train/X_1000_5sec_v0.pkl', 'rb')
X_training_v0 = pickle.load(file)
file.close()

file = open('XY_train/y_1000_5sec_v0.pkl', 'rb')
y_training_v0 = pickle.load(file)
file.close()

#v1
file = open('XY_train/X_1000_5sec_v1.pkl', 'rb')
X_training_v1 = pickle.load(file)
file.close()

file = open('XY_train/y_1000_5sec_v1.pkl', 'rb')
y_training_v1 = pickle.load(file)
file.close()

#v2
file = open('XY_train/X_1000_5sec_v2.pkl', 'rb')
X_training_v2 = pickle.load(file)
file.close()

file = open('XY_train/y_1000_5sec_v2.pkl', 'rb')
y_training_v2 = pickle.load(file)
file.close()

#v3
file = open('XY_train/X_1000_5sec_v3.pkl', 'rb')
X_training_v3 = pickle.load(file)
file.close()

file = open('XY_train/y_1000_5sec_v3.pkl', 'rb')
y_training_v3 = pickle.load(file)
file.close()

#v4
file = open('XY_train/X_1000_5sec_v4.pkl', 'rb')
X_training_v4 = pickle.load(file)
file.close()

file = open('XY_train/y_1000_5sec_v4.pkl', 'rb')
y_training_v4 = pickle.load(file)
file.close()

#v5
file = open('XY_train/X_1000_5sec_v5.pkl', 'rb')
X_training_v5 = pickle.load(file)
file.close()

file = open('XY_train/y_1000_5sec_v5.pkl', 'rb')
y_training_v5 = pickle.load(file)
file.close()

#v6
file = open('XY_train/X_1000_5sec_v6.pkl', 'rb')
X_training_v6 = pickle.load(file)
file.close()

file = open('XY_train/y_1000_5sec_v6.pkl', 'rb')
y_training_v6 = pickle.load(file)
file.close()

#v7
file = open('XY_train/X_1000_5sec_v7.pkl', 'rb')
X_training_v7 = pickle.load(file)
file.close()

file = open('XY_train/y_1000_5sec_v7.pkl', 'rb')
y_training_v7 = pickle.load(file)
file.close()

X_training = np.concatenate([X_training_v0, X_training_v1, X_training_v2, X_training_v3,
                            X_training_v4, X_training_v5, X_training_v6, X_training_v7])
y_training = np.concatenate([y_training_v0, y_training_v1, y_training_v2, y_training_v3,
                            y_training_v4, y_training_v5, y_training_v6, y_training_v7])

del(X_training_v0)
del(X_training_v1)
del(X_training_v2)
del(X_training_v3)
del(X_training_v4)
del(X_training_v5)
del(X_training_v6)
del(X_training_v7)

# load validation features and labels
file = open('XY_val/X_400_5sec_100_100_200.pkl', 'rb')
X_val = pickle.load(file)
file.close()

file = open('XY_val/y_400_5sec_100_100_200.pkl', 'rb')
y_val = pickle.load(file)
file.close()

# one-hot encode labels
y_training_oh = encode_and_oh_labels(y_training)
y_val_oh = encode_and_oh_labels(y_val)

print('Size of X_training: {}'.format(X_training.shape))
print('Size of X_val: {}'.format(X_val.shape))

Size of X_training: (8000, 2754, 101)
Size of X_val: (400, 2754, 101)


In [7]:
label_decoder_dict = {0:'empty', 1:'enough', 2:'not_enough'}

In [8]:
# shuffle training data explicitly
np.random.seed(134)
perm = np.random.permutation(len(X_training))

X_training = X_training[perm]
y_training = y_training[perm]
y_training_oh = y_training_oh[perm]

In [25]:
def model(input_shape):
    X_input = Input(shape = input_shape)
    
    X = Conv1D(filters=196, kernel_size=15, strides=4)(X_input)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = Dropout(0.0)(X)
    
    X = Bidirectional(LSTM(units=128))(X)
    X = Dense(3, activation='softmax')(X)

    model = Model(inputs = X_input, outputs = X)
    
    return model  

In [9]:
def gru_model(input_shape):
    X_input = Input(shape = input_shape)
    
    X = Conv1D(filters=196, kernel_size=15, strides=4)(X_input)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = Dropout(0.5)(X)

    X = GRU(units=128, return_sequences = True)(X)
    X = Dropout(0.5)(X)
    X = BatchNormalization()(X)
    
    X = GRU(units=128, return_sequences = True)(X)
    X = Dropout(0.5)(X)
    X = BatchNormalization()(X)
    X = Dropout(0.5)(X)
    
    X = TimeDistributed(Dense(1, activation='relu'))(X)
    X = Reshape((685,))(X)
    X = Dense(3, activation='softmax')(X)
    
    model = Model(inputs=X_input, outputs=X)
    return model

In [9]:
def gru_model2(input_shape):
    X_input = Input(shape = input_shape)
    
    X = Conv1D(filters=196, kernel_size=15, strides=4)(X_input)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = Dropout(0.5)(X)

    X = GRU(units=128, return_sequences = True)(X)
    X = Dropout(0.5)(X)
    X = BatchNormalization()(X)
    
    X = GRU(units=128, return_sequences = True)(X)
    X = Dropout(0.5)(X)
    X = BatchNormalization()(X)
    X = Dropout(0.5)(X)
    
#    X = TimeDistributed(Dense(1, activation='relu'))(X)
#    X = Reshape((685,))(X)
    X = Flatten()(X)
    X = Dense(3, activation='softmax')(X)
    
    model = Model(inputs=X_input, outputs=X)
    return model

In [18]:
def gru_model3(input_shape):
    X_input = Input(shape = input_shape)
    
    X = Conv1D(filters=196, kernel_size=15, strides=4)(X_input)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = Dropout(0.5)(X)

    X = GRU(units=128, return_sequences = True)(X)
    X = Dropout(0.5)(X)
    X = BatchNormalization()(X)
    
    X = GRU(units=128, return_sequences = False)(X)
    X = Dropout(0.5)(X)
    X = BatchNormalization()(X)
    X = Dropout(0.5)(X)
    
#    X = TimeDistributed(Dense(1, activation='relu'))(X)
#    X = Reshape((685,))(X)
    #X = Flatten()(X)
    X = Dense(3, activation='softmax')(X)
    
    model = Model(inputs=X_input, outputs=X)
    return model

In [10]:
K.clear_session()

In [11]:
model = gru_model(input_shape = (Tx, n_freq))



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2754, 101)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 685, 196)          297136    
_________________________________________________________________
batch_normalization_1 (Batch (None, 685, 196)          784       
_________________________________________________________________
activation_1 (Activation)    (None, 685, 196)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 685, 196)          0         
_________________________________________________________________
gru_1 (GRU)                  (None, 685, 128)          124800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 685, 128)          0         
__________

In [13]:
import keras

In [16]:
import tensorflow as tf

In [19]:
def custom_loss_w_label_smoothing(y_true, y_pred):
    return tf.keras.losses.categorical_crossentropy(y_true, y_pred, label_smoothing=0.2)

In [20]:
version = '{}'.format(int(time()%1e7))
tb_path = 'tensorboard_logs/{}'.format(version)
tensorboard = TensorBoard(log_dir=tb_path, histogram_freq=1)
print('version: {}'.format(version))

#opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss=custom_loss_w_label_smoothing, optimizer='adam', metrics=["accuracy"])

version: 8093344



In [21]:
model.fit(X_training, y_training_oh, batch_size=128, epochs=30, shuffle=True,
         validation_data = [X_val, y_val_oh], callbacks = [tensorboard])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 8000 samples, validate on 400 samples








Epoch 1/30
8000/8000 [==============================] - 688s 86ms/step - loss: 1.1239 - acc: 0.5456 - val_loss: 0.9560 - val_acc: 0.5625

Epoch 2/30
8000/8000 [==============================] - 861s 108ms/step - loss: 0.9513 - acc: 0.5743 - val_loss: 1.1044 - val_acc: 0.2550
Epoch 3/30
8000/8000 [==============================] - 1071s 134ms/step - loss: 0.8990 - acc: 0.5907 - val_loss: 1.1089 - val_acc: 0.2525
Epoch 4/30
8000/8000 [==============================] - 933s 117ms/step - loss: 0.8750 - acc: 0.6118 - val_loss: 1.1012 - val_acc: 0.2975
Epoch 5/30
8000/8000 [==============================] - 1062s 133ms/step - loss: 0.7973 - acc: 0.7139 - val_loss: 1.0449 - val_acc: 0.3975
Epoch 6/30
8000/8000 [==============================] - 958s 120ms/step - loss: 0.7319 - acc: 0.7768 - val_loss: 1.1630 - val_acc: 0.2525
Epoch 7/30
8000/80

In [22]:
# get predictions on validation set
labels = y_val
preds = model.predict(X_val)
preds_text = [label_decoder_dict[v] for v in np.argmax(preds, axis=1)]

In [36]:
# get predictions on training set
labels = y_training[:200]
preds = model.predict(X_training[:200])
preds_text = [label_decoder_dict[v] for v in np.argmax(preds, axis=1)]

In [23]:
results = pd.DataFrame({'labels':labels, 'preds':preds_text})

In [24]:
pd.crosstab(results['labels'], results['preds'])

preds,empty,enough,not_enough
labels,,,
empty,11,13,176
enough,0,99,1
not_enough,0,8,92


In [25]:
model.save('trained_models/{}.h5'.format(version))

In [26]:
version

'8093344'

####  version notes

- 1st: good data, 1000.  perfect on train and test but overfitting. randomizing where words appear in clip but only using a few dozen clips.
- 2nd: good data, 4000. perfect on train and test but overfitting. same as above.

add data augmentation.

- `4055836`: train on 1000 using `gru_model2`, after (4x4) data augmentation on each of enoughs, not_enoughs, empty. high 90s, except for predicting some not_enoughs as enoughs.
- `4058922`: train on 4000 using `gru_model2` for 2 epochs.  val set pulls from different folders of augmented clips (but same original 10 clips).

add train/val split at original recorded_clip level.

- `4106576`: train on 1000 using LSTM `model` for 20 epochs.  data clips were split into 8/10 for train and 2/10 for val, and then augmented 5x5 each, with ps sigma = 2.5/2 and ts sigma = 0.4/2.  was perfect on both train and val!
- `4109505`: same as previous, but 5 epochs with 4000 training samples. also basically perfect on train and val.
- `4192521`: 4000 samples with 50+ 'collected' backgrounds. 10 epochs. overfits around 8th.  predicts 'enough' too often but does pretty good with 'not enough'
- `4206950`: 8000 samples like previous, but batch size = 128 instead of 64.

switch to new 'enough' samples (k.), and add some more 'not enough' samples.

- `4909928`: train on 1000 using LSTM `model` for 5 epochs, with new different 'enough' samples.  seemed to struggle so trying `gru_model`.
- `4206950`: 1000 for 10 epochs using `gru_model`. does perfect on empty and enough, but thinks 92% of the not_enough's are empty.
- `4913454`: 4000 for 5 epochs using `gru_model`. 
- `4943569`: same but for 20 epochs.  did almost perfect on train and val, but during runtime only predicts not_enough.  needs more noisy backgrounds, and maybe less epochs training so it generalizes better.

record 720 more background clips.
- `6782372`: 4000 for 10 epochs using `gru_model`. started to improve after 6 epochs so i'll retrain for 20 total. 
- `6788415`: 4000 for 20 epochs, didn't do much better.

switch class mix from 333/333/334 to 200/200/600 (enough/not_enough/empty)

- `6860093`: 8000 for 10 epochs with the new mix.  still `gru_model`. pretty bad so not saving for runtime testing.
- `6880363`: 8000 for 100 epochs. 98% recall on 'empty', 100% on 'enough', and 94% on 'not_enough' on val set. no good during runtime.
- `8093344`: 8000 for 30 epochs, but added custom loss with label_smoothing=0.2. does well on enough and not_enough, but horrible on empties (on val).